Install the OpenAI libray

In [31]:
%pip install openai  python-dotenv



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Load neccesary environment variables

In [44]:
# Import the OpenAI API key from the .env file
import os # Import the OS module
from dotenv import load_dotenv # Import the load_dotenv function from the dotenv module
load_dotenv() # Load the .env file

True

Get variables from the environment

In [46]:
# Get the OpenAI API key from the environment
openai_api_key = os.getenv('OPEN_AI_API_KEY') # Get the OpenAI API key from the environment



Create connection to OpenAI

In [47]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key) # Create a new OpenAI client

Initiate ChatGPT assistant

In [ ]:
assistant = client.Assistant.create(
    name="Kudu AI",
    instructions="You are a medical assistant and helps patients with questions around their medication. You can provide information about the medication, side effects, and how to take it. You can also help patients find the right medication for their condition.", # Provide instructions to the AI
    tools=[{"type": ""}], # Define the tools that the AI can use
    completion={"model": "gpt-4o", "chat": True}, # Use the GPT-4 model and enable chat mode
    )